## Basic Callbacks

For the latest code revision, fork the code repo: https://github.com/mannyjrod/Dash_Apps_Sandlot

Author: Emmanuel Rodriguez

[emmanueljrodriguez.com/](https://emmanueljrodriguez.com/)

Date: 19NOV2023

Ref: https://dash.plotly.com/basic-callbacks

Ref: Schroeder, A. *The Book of Dash*, p. 61

"A *Dash callback* enables user interactivity within the dashboard app; it is the mechanism that connects the Dash components to each other so that performing one action causes something else to happen.

E.g., When the app user selects a dropdown value, the figure is updated; when the user clicks a button, the color of the app's title changes or another graph is added to the page. I.e., Without callbacks, the app is static and the user cannot modify anything."

The Dash callback has two parts:

1. The **callback decorator** that identifies the relevant components, defined in the layout section:

`@app.callback()`

2. The **callback function** that defines how those Dash components should interact:

`def function_name(y):
    return x`

Defined by dash.plotly as:

"*Callback functions:* functions that are automatically called by Dash whenever an input component's property changes, in order to update some property in another component (the output)."

Ref: https://dash.plotly.com/basic-callbacks

### Simple Interactive Dash App

In [1]:
# Load packages
from dash import Dash, dcc, html, Input, Output, callback

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),
])

In [6]:
# Callback decorator - recall: this identifies the relevant components defined in the above layout section
@callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value')
)

# Define the callback function - which defines how those Dash components should interact.
def update_output_div(input_value):
    return f'Output: {input_value}'

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 2450, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 1867, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 1945, in full_dispatch_request
    self.try_trigger_before_first_request_functions()
  File "C:\Users\ejoaq\anaconda3\Lib\site-packages\flask\app.py", line 1993, in try_trigger_before_first_request_functions
    func()
  File "C:\Users\ejoaq\anaconda3\L

Dash Callbacks are a form of *Reactive Programming*; the outputs *react* to changes in the inputs automatically.

"It's sort of like programming with Microsoft Excel: whenever a cell changes (the input), all the cells that depend on that cell (the outputs) will get updated automatically."

### Dash App Layout With Figure and Slider

In [7]:
# Import packages
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

import pandas as pd

# Load data - note: The dataframe df is in the global state of the app and can be read inside the callback function.
# This ensures the operation is only done once (loading data into memory can be expensive) -- when the app server starts.
# Expensive initializations such as these, should be done in the global scope of the app.
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

# Initialize app
app = Dash(__name__)

# App layout configuration
app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])

# Callback decorator - remember: The callback decorator registers the callback function with the Dash app, this 'tells'
# the app when to call the function and how to use the return value of the function to update the app.

# In this scenario, the app will call the callback function when the value of the year-slider changes, and the figure
# graph-with-slider will change accordingly.
@callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))

# Callback function - wrapped by the callback decorator, holds the logic that will be executed when the function is called.

# In this scenario, 'update_figure' is the name of the function, with the input argument set to 'selected_year'.
# The df object year is set to the input argument, which is then used as the variable to slice the df, which is then set 
# to the 'filtered_df' variable. This *new* df is used to create a scatter plot.

# Note: The callback does not modify the original data, it only creates copies of the dataframe ('filtered_df') by
# filtering using pandas. -- Callbacks should NEVER modify variables outside of their scope, such as a global variable.
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]
    
    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                    size="pop", color="continent", hover_name="country",
                    log_x=True, size_max=55)
    
    fig.update_layout(transition_duration=500) # the `layout.transition` object is set to 500 ms (?), essentially setting
    # the chart to update from one state to the next smoothly, as if it were animated.
    
    return fig

if __name__  == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
